# MUHAMMAD HASNAN RAMADHAN

## Import Library

In [ ]:
import tensorflow as tf
print(tf.__version__)
import os
import zipfile
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import files
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Import Dataset

In [ ]:
!wget --no-check-certificate \
      https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip \
      -O /tmp/rockpaperscissors.zip

In [ ]:
# melakukan ekstraksi pada file zip
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
base_dir = '/tmp/rockpaperscissors'
train_dir = os.path.join(base_dir, 'rps-cv-images')
validation_dir = os.path.join(base_dir, 'rps-cv-images')

In [ ]:
os.listdir('/tmp/rockpaperscissors/rps-cv-images')

In [ ]:
# Directory for training pictures
train_rock_dir = os.path.join(train_dir, 'rock')
train_paper_dir = os.path.join(train_dir, 'paper')
train_scissors_dir = os.path.join(train_dir, 'scissors')

In [ ]:
# Directory for validation picture
val_rock_dir = os.path.join(validation_dir, 'rock')
val_paper_dir = os.path.join(validation_dir, 'paper')
val_scissors_dir = os.path.join(validation_dir, 'scissors')

## Images Augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                rescale=1.0/255,
                rotation_range=20,
                horizontal_flip=True,
                shear_range = 0.2,
                fill_mode = 'nearest',
                validation_split=0.4)

test_datagen = ImageDataGenerator(
                        rescale=1./255)

# validation_datagen = ImageDataGenerator(rescale = 1.0/255,
#                                         validation_split=0.4)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(100,150),
        batch_size=32,
        class_mode='categorical',
        subset='training')

validation_generator = train_datagen.flow_from_directory(
        validation_dir,
        target_size=(100,150),
        batch_size=32,
        class_mode='categorical',
        subset='validation')

## Construct a model for CNN

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', input_shape=(100, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

print(model.summary())

## Compile a model

In [ ]:
# compile model dengan 'nadam' optimizer and loss function 'categorical_crossentropy'
from tensorflow.keras.optimizers import Nadam

model.compile(loss='categorical_crossentropy',
              optimizer = Nadam(lr=0.001),
              metrics=['accuracy'])

## Train a model using model.fit

In [ ]:
# latih model dengan model.fit
history = model.fit(
      train_generator,
      steps_per_epoch=20,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=17, # tambahkan epochs jika akurasi model belum optimal
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,  # berapa batch yang akan dieksekusi pada setiap epoch
      verbose=1,
  )

## Evaluate the model

In [ ]:
evaluation = model.evaluate(train_generator)
print("Accuracy: {:.4f}".format(evaluation[1]))
print("Loss: {:.4f}".format(evaluation[0]))

In [ ]:
evaluation_2 = model.evaluate(validation_generator)
print("Accuracy: {:.4f}".format(evaluation_2[1]))
print("Loss: {:.4f}".format(evaluation_2[0]))

## Visualize accuracy of the result of the model

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# Plot Akurasi
plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.figure()

# Plot Loss
plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

## TEST THE MODEL WITH YOUR OWN PHOTOS HERE!

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = fn
  img = image.load_img(path, target_size=(100,150))

  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])

  classes = model.predict(images, batch_size=25)
  print(fn)
  if classes[0,0] == 1:
    print("It's PAPER!")
  elif classes[0,1] == 1:
    print(("It's ROCK!"))
  elif classes[0,2] == 1:
    print("It's Scissors!")